# to compute masses of coding polymers

*M-A Delsuc - V2 - 12 Nov 2020*

- V1: initial (w or w/o Lau notation)
- V2: 
    - adding structure computation
    - a cleaner mass table
    - some computation on the delta m/z
    - requires to download [github.com/delsuc/isotope](https://github.com/delsuc/isotope)

*to download isotope, simply remove the # in the next cell and execute it* (only once !)

In [1]:
#!git clone https://github.com/delsuc/isotope.git

In [2]:
# The problem is equivalent to enumerating all possible drawings of a set of equivalent dices (no ordereing).

# first define the `enumerateDice(P, N)` function which returns all the drawing of P dices with N faces

def enumerateDice(P, N):
    """
    enumerates all the drawing of N dices with P faces
    (no order)
    """
    from math import factorial as f
    from copy import copy

    """
    simulated as marbles located in a frame of P columns with N entries,
    where each column cannot have more marbles than the previous one
    (so ordered from high to low)
    add one on 1st col
    if col<=N
        keep
    else
        add one in next empty:  (N=3, P=7)
            *                  * 
            * * * *            * * * * *
            * * * * * * *   => * * * * * * *

    """
    Debug = False
    theory = f(N+P-1)//(f(P)*f(N-1))
    if Debug:
        print('à trouver: {} dans {}'.format(theory, N**P ) )

    state = [1]*P  # start with one in every col
    yield state

    count = 1
    
    done = False
    while not done:
        if Debug: print(state)

        # add one:
        if state[0]<N:
            state[0] += 1
        else:             # search for adding
            for i in range(1,P):
                if state[i] < state[i-1]:
                    state[i] += 1
                    for j in range(0,i):
                        state[j] = state[i]
                    if Debug: print('#',state)
                    break
        count += 1
        yield state
        if sum(state) == N*P:
            done = True
    if count != theory:
        print('** ERROR ** something went wrong in the code - please verify results !')

In [3]:
# example: combinations for 3 dices with 4 faces
# or an unordered polymer of 3 monomers to take in 4 different species
for i in enumerateDice(3, 4):
    print(i)

[1, 1, 1]
[2, 1, 1]
[3, 1, 1]
[4, 1, 1]
[2, 2, 1]
[3, 2, 1]
[4, 2, 1]
[3, 3, 1]
[4, 3, 1]
[4, 4, 1]
[2, 2, 2]
[3, 2, 2]
[4, 2, 2]
[3, 3, 2]
[4, 3, 2]
[4, 4, 2]
[3, 3, 3]
[4, 3, 3]
[4, 4, 3]
[4, 4, 4]


## define parameters

In [4]:
N = 8
P = 8
#monomasses = [138.0082,152.0238,166.0395,180.0551,194.0708,208.0864,222.1021,250.1334]
#tagmasses = [149.0966,690.2232,681.2329,790.1492,730.2293,799.1132,751.1271,712.2387,696.2438,672.2326,476.1798]
charge = -3

# order of the tag in the polymer
tagorder = ['None', "D", "P", "R", "F", "I", "B", "G", "A", "C", "T"]

In [5]:
# isotope is found at github.com/delsuc/isotope
import isotope.isotopes as iso
from isotope.isotopes import parse_formula as form

# compute masses (counting the phosphate on the right)

# monomers
M1 = form("(CH2)3 HPO4")  # monomer M1
M1mas = M1.monoisotop()
CH2 = form("CH2").monoisotop()
monomasses = [M1mas+n*CH2 for n in range(N)]
monomasses[N-1] += CH2    # M8 has one additional CH2

# spacer  omega-alpha
alpha = form("ON C5H5 (CH3)4 PO4H").monoisotop()  # right part of the breaking bond
omega = form("(CH2)2 C6H4 CH CH3").monoisotop()   # left part of the breaking bond

# left and right ends
leftend = form("OH").monoisotop()  
rightend = 0 #form("OH").monoisotop()

tags = {}
phospho = form("PO4H CH2 (CH O CH CH2 CH)")
tags["None"] =                0.0 
tags["P"] = (phospho + form("N4 C5 H3")).monoisotop()
tags["A"] = (phospho + form("N5 C5 H4")).monoisotop()
tags["G"] = (phospho + form("N5 O C5 H4")).monoisotop()
tags["E"] = (phospho + form("N5 C6 H4")).monoisotop()
tags["F"] = tags["G"] + form("F").monoisotop() - form("H").monoisotop()
tags["C"] = (phospho + form("N3 O C4 H4")).monoisotop()
tags["D"] = tags["C"] + form("F").monoisotop() - form("H").monoisotop() 
tags["R"] = tags["G"] + form("Br").monoisotop() - form("H").monoisotop()
tags["B"] = (phospho + form("N2 O2 C4 Br H2")).monoisotop()
tags["I"] = (phospho + form("N2 O2 C4 I H2")).monoisotop()
tags["T"] = (form("OH CH2 (CH O CH CH2 CH) N2 O2 C5 H5")).monoisotop()

# then add alpha and omega to all inter-tags
for i,t in enumerate(tagorder):
    if i == 0:
        tags[t] += leftend + omega
    elif i == len(tagorder)-1:
        tags[t] += alpha + rightend
    else:
        tags[t] += alpha + omega
tags

{'None': 149.09664003647,
 'P': 681.23286579447,
 'A': 696.2437648313401,
 'G': 712.2386794509,
 'E': 325.05760513664,
 'F': 730.22925763883,
 'C': 672.2325314413,
 'D': 690.2231096292301,
 'R': 790.1491915188299,
 'B': 751.12705909192,
 'I': 799.11319499192,
 'T': 476.17979121461}

### utilities

In [6]:
def enumpoly():
    "compute the m/z from the enumerated list of polymers"
    mH = form("H").monoisotop()
    for values in enumerateDice(P,N):
        massbase = sum([ monomasses[n-1] for n in values])
        mz = [massbase + tags[tag] for tag in tagorder]
        if charge <0:
            mz = [(m/(-charge))-mH+iso.m_e for m in mz]
        else:
            mz = [m/charge+mH-iso.m_e  for m in mz]
        yield (values, mz)
def poly2lau(poly, N):
    "code the 'Laurence' format from polymer - (i.e. nub of each possible monomer)"
    lau = []
    for n in range(N):
        lau.append(sum( [1 for p in poly if p==(n+1)]))
    return lau
#    return [sum( [1 for p in poly if p==(n+1)]) for n in range(N)]
# check
poly2lau([6,5,4,3,2,1,1,1],8)

[3, 1, 1, 1, 1, 1, 0, 0]

## just list the values

In [7]:
for v,m in enumpoly():
    s = "".join([str(i) for i in v])
    print(s,m)

11111111 [416.7134577806091, 597.0889476448624, 594.0921996999424, 630.3976416080624, 610.4243303147291, 633.385642765759, 617.3902641324257, 604.4274709187524, 599.0958327122324, 591.0920882488858, 525.7411748399891]
21111111 [421.38534113532245, 601.7608309995758, 598.7640830546558, 635.0695249627757, 615.0962136694425, 638.0575261204724, 622.0621474871391, 609.0993542734658, 603.7677160669458, 595.7639716035992, 530.4130581947024]
31111111 [426.05722449003576, 606.4327143542891, 603.435966409369, 639.741408317489, 619.7680970241557, 642.7294094751858, 626.7340308418525, 613.771237628179, 608.4395994216591, 600.4358549583123, 535.0849415494158]
41111111 [430.7291078447491, 611.1045977090024, 608.1078497640824, 644.4132916722024, 624.439980378869, 647.401292829899, 631.4059141965657, 618.4431209828924, 613.1114827763724, 605.1077383130257, 539.7568249041291]
51111111 [435.40099119946245, 615.7764810637158, 612.7797331187958, 649.0851750269157, 629.1118637335824, 652.0731761846124, 636

74433311 [500.80735816544905, 681.1828480297024, 678.1861000847823, 714.4915419929023, 694.518230699569, 717.479543150599, 701.4841645172656, 688.5213713035923, 683.1897330970725, 675.1859886337257, 609.8350752248291]
84433311 [510.1511248748758, 690.5266147391292, 687.529866794209, 723.835308702329, 703.8619974089958, 726.8233098600258, 710.8279312266924, 697.8651380130191, 692.5334998064992, 684.5297553431525, 619.1788419342558]
55433311 [496.1354748107357, 676.510964674989, 673.5142167300689, 709.8196586381889, 689.8463473448556, 712.8076597958857, 696.8122811625524, 683.8494879488791, 678.517849742359, 670.5141052790124, 605.1631918701157]
65433311 [500.80735816544905, 681.1828480297024, 678.1861000847823, 714.4915419929023, 694.518230699569, 717.479543150599, 701.4841645172656, 688.5213713035923, 683.1897330970725, 675.1859886337257, 609.8350752248291]
75433311 [505.4792415201624, 685.8547313844157, 682.8579834394957, 719.1634253476157, 699.1901140542824, 722.1514265053124, 706.15

88888821 [645.6357421615623, 826.0112320258156, 823.0144840808957, 859.3199259890157, 839.3466146956823, 862.3079271467124, 846.3125485133792, 833.3497552997057, 828.0181170931857, 820.014372629839, 754.6634592209423]
33333331 [482.1198247465957, 662.495314610849, 659.498566665929, 695.804008574049, 675.8306972807156, 698.7920097317457, 682.7966310984125, 669.8338378847391, 664.5021996782191, 656.4984552148724, 591.1475418059757]
43333331 [486.7917081013091, 667.1671979655624, 664.1704500206424, 700.4758919287624, 680.502580635429, 703.463893086459, 687.4685144531256, 674.5057212394524, 669.1740830329325, 661.1703385695857, 595.8194251606891]
53333331 [491.46359145602247, 671.8390813202758, 668.8423333753558, 705.1477752834758, 685.1744639901424, 708.1357764411724, 692.1403978078392, 679.1776045941658, 673.8459663876458, 665.8422219242991, 600.4913085154025]
63333331 [496.13547481073584, 676.5109646749892, 673.5142167300692, 709.8196586381891, 689.8463473448558, 712.8076597958858, 696.

76653322 [538.1824250031558, 718.5579148674092, 715.5611669224892, 751.8666088306092, 731.8932975372759, 754.8546099883058, 738.8592313549725, 725.8964381412991, 720.5647999347791, 712.5610554714324, 647.2101420625359]
86653322 [547.5261917125824, 727.9016815768359, 724.9049336319158, 761.2103755400358, 741.2370642467024, 764.1983766977324, 748.202998064399, 735.2402048507257, 729.9085666442058, 721.9048221808591, 656.5539087719625]
77653322 [542.8543083578691, 723.2297982221223, 720.2330502772024, 756.5384921853224, 736.565180891989, 759.5264933430191, 743.5311147096859, 730.5683214960125, 725.2366832894924, 717.2329388261458, 651.8820254172491]
87653322 [552.1980750672958, 732.5735649315491, 729.5768169866292, 765.8822588947492, 745.9089476014158, 768.8702600524458, 752.8748814191126, 739.9120882054392, 734.5804499989191, 726.5767055355725, 661.2257921266759]
88653322 [561.5418417767224, 741.9173316409758, 738.9205836960557, 775.2260256041757, 755.2527143108424, 778.2140267618724, 76

64444333 [524.1667749390159, 704.5422648032692, 701.5455168583492, 737.8509587664691, 717.8776474731359, 740.8389599241659, 724.8435812908326, 711.8807880771593, 706.5491498706392, 698.5454054072926, 633.194491998396]
74444333 [528.8386582937292, 709.2141481579824, 706.2174002130624, 742.5228421211824, 722.5495308278491, 745.5108432788791, 729.5154646455459, 716.5526714318726, 711.2210332253524, 703.2172887620059, 637.8663753531091]
84444333 [538.1824250031559, 718.5579148674092, 715.5611669224892, 751.8666088306092, 731.8932975372759, 754.8546099883058, 738.8592313549726, 725.8964381412993, 720.5647999347791, 712.5610554714326, 647.2101420625359]
55444333 [524.1667749390159, 704.5422648032692, 701.5455168583492, 737.8509587664691, 717.8776474731359, 740.8389599241659, 724.8435812908326, 711.8807880771593, 706.5491498706392, 698.5454054072926, 633.194491998396]
65444333 [528.8386582937292, 709.2141481579824, 706.2174002130624, 742.5228421211824, 722.5495308278491, 745.5108432788791, 72

## export as a csv file 

In [8]:
filename = "prev_triades_V2.csv"
with open(filename,'w') as F:
    print("# table generated with {} monomers and {} positions charge={}".format(N,P,charge), file=F)
    print("polymer",'000,001,010,011,100,101,110,111', *tagorder, sep=',',file=F)
    for v,m in enumpoly():
        s = "".join([str(i) for i in v])
        print(s, *poly2lau(v,N), *m, sep=',',file=F)


## reverse table
and compute a few parameters

In [9]:
from collections import defaultdict
revtable = defaultdict(set)
for values,masses in enumpoly():
    for i,m in enumerate(masses):
        s = str(sum([v for v in values])) + tagorder[i]
        revtable[round(m,5)].add(s)

masses = sorted(list(revtable.keys()))
diff = [masses[i]-masses[i-1] for i in range(1,len(masses))]
meandiff = sum(diff)/len(diff)
print('{} different possible m/z from {} to {} - mean Δm/z {:.3f} - smallest Δm/z: {:.3f}'.format(len(masses), min(masses),max(masses), meandiff, min(diff)) )
print('the number is the sum of the monomer indices')
for m in sorted(revtable.keys()):
    print(m, sorted(revtable[m]))

704 different possible m/z from 416.71346 to 932.38618 - mean Δm/z 0.734 - smallest Δm/z: 0.055
the number is the sum of the monomer indices
416.71346 ['8None']
421.38534 ['9None']
426.05722 ['10None']
430.72911 ['11None']
435.40099 ['12None']
440.07287 ['13None']
444.74476 ['14None']
449.41664 ['15None']
454.08852 ['15None', '16None']
458.76041 ['16None', '17None']
463.43229 ['17None', '18None']
468.10417 ['18None', '19None']
472.77606 ['19None', '20None']
477.44794 ['20None', '21None']
482.11982 ['21None', '22None']
486.79171 ['22None', '23None']
491.46359 ['22None', '23None', '24None']
496.13547 ['23None', '24None', '25None']
500.80736 ['24None', '25None', '26None']
505.47924 ['25None', '26None', '27None']
510.15112 ['26None', '27None', '28None']
514.82301 ['27None', '28None', '29None']
519.49489 ['28None', '29None', '30None']
524.16677 ['29None', '30None', '31None']
525.74117 ['8T']
528.83866 ['29None', '30None', '31None', '32None']
530.41306 ['9T']
533.51054 ['30None', '31None', '

## and export as a csv file

In [10]:
filename = "sorted_masses_V2.csv"
with open(filename,'w') as F:
    print("# table generated with {} monomers and {} positions charge={}".format(N,P,charge), file=F)
    print('{} different possible m/z from {} to {} - mean Δm/z {:.3f} - smallest Δm/z: {:.3f}'.format(
        len(masses), min(masses),max(masses), meandiff, min(diff)), file=F )
    print('the number is the sum of the monomer indices, the letter is the tag',file=F)
    print('mass', 'polymers', sep=',',file=F)
    for m in sorted(revtable.keys()):
        print(m, *revtable[m], sep=',',file=F)



*Marc-André*